In [ ]:
import os
from langchain_openai import AzureChatOpenAI

api_key = os.environ["AZURE_OPENAI_KEY"]
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
"https://gpt-amayuelas.openai.azure.com/"

llm = AzureChatOpenAI(
    api_key=api_key,
    azure_endpoint=endpoint,
    azure_deployment="gpt-4o",  # or your deployment
    api_version="2024-12-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

print(ai_msg.content)

J'adore la programmation.


In [10]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
llm.invoke(messages)

AIMessage(content='Hello, world! 🌍 How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 11, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BSAYi2U6ltzax5lB07Ppj3qqX8DTf', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_mat

### Using Talivy RAG Example (Use for online search tool)

In [6]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")
_set_env("TAVILY_API_KEY")
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("How do you play catan?")
search_docs

[{'title': 'How to Play CATAN - Asmodee UK',
  'url': 'https://www.asmodee.co.uk/blogs/news/how-to-play-catan',
  'content': 'The resource cards should be separated into their 5 different piles, face up next to the board. Development cards should be shuffled and placed face down by the board too, where they are accessible to players.\n\nPlaying the Game\n\n\n\nCATAN has three main aspects of gameplay: producing resources, trading resources, and using the resources to build. When a player takes their turn, they will move through each of these three actions. [...] Can you play CATAN with 2 people?\n\nYou can play CATAN\xa0with two people if you have the core game. Simply place two sets of game pieces on the board to act as imaginary players. Instead of stealing cards when using the robber, this will provide resources.\n\nIs it better to go first or last in CATAN? [...] CATAN, otherwise known as Settlers of CATAN, is a popular board game that combines strategy, negotiation, and a dash of 

## Tool Calling ReAct Agent with Memory

In [16]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)


# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
memory = MemorySaver()
react_graph = builder.compile(checkpointer=memory)


# Specify a thread
config = {"configurable": {"thread_id": "1"}}


# First LLM interaction
messages = [HumanMessage(content="Add 3 and 4.")]
messages = react_graph.invoke({"messages": messages},config)
for m in messages['messages']:
    m.pretty_print()


messages = [HumanMessage(content="There are a few things I want you to do. First, take the last output and add 2. Multiply the output by 2. Actually, before multiplying I want you to add the output to 1. Lastly, divide the output by 5")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================
Tool Calls:
  add (call_UZxkuuvODjvK5D6Iwwq7ERHI)
 Call ID: call_UZxkuuvODjvK5D6Iwwq7ERHI
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.
================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================
Tool Calls:
  add (call_UZxkuuvODjvK5D6Iwwq7ERHI)
 Call ID: call_UZxkuuvODjvK5D6Iwwq7ERHI
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.
==============

## Basic Catan Setup